In [ ]:
import numpy as np
import pandas as pd
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

sns.set_style("darkgrid")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
movie_title = pd.read_csv('/kaggle/input/movietitle/trimdown_movieTitle.csv')

In [ ]:
cust_id = pd.read_csv('/kaggle/input/cust-id/trimdown.csv')

In [ ]:
min_movie_ratings = 10000
filter_movies = (cust_id['Movie_Id'].value_counts()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

# Filter sparse users
min_user_ratings = 200
filter_users = (cust_id['Cust_Id'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Actual filtering
df_filterd = cust_id[(cust_id['Movie_Id'].isin(filter_movies)) & (cust_id['Cust_Id'].isin(filter_users))]
del filter_movies, filter_users, min_movie_ratings, min_user_ratings
print('Shape User-Ratings unfiltered:\t{}'.format(cust_id.shape))
print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))

In [ ]:
cust_id = cust_id.drop('Unnamed: 0', axis=1)

In [ ]:
mt = movie_title['Name']

In [ ]:
mt.reset_index()

In [ ]:
reader = Reader()

data = Dataset.load_from_df(cust_id[['Cust_Id', 'Rating', 'Movie_Id']][:100000], reader)

svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

In [ ]:
df_785314 = cust_id[(cust_id['Cust_Id'] == 785314) & (cust_id['Rating'] == 5)]
df_785314 = df_785314.set_index('Movie_Id')
df_785314 = df_785314.join(mt)['Name']
print(df_785314)

In [ ]:
user_785314 = mt.copy()
user_785314 = user_785314.reset_index()
user_785314 = user_785314[~user_785314['Movie_Id'].isin(drop_movie_list)]

data = Dataset.load_from_df(cust_id[['Cust_Id', 'Movie_Id', 'Rating']], reader)

trainset = data.build_full_trainset()
svd.train(trainset)

user_785314['Estimate_Score'] = user_785314['Movie_Id'].apply(lambda x: svd.predict(785314, x).est)

user_785314 = user_785314.drop('Movie_Id', axis = 1)

user_785314 = user_785314.sort_values('Estimate_Score', ascending=False)
print(user_785314.head(10))